In [1]:
from datetime import datetime

import numpy as np
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset

from datasets.CustomTrainingImageDataset import generate_balanced_dataset
from models.cnn import CnnModel
from train import train_classifier
from utils import plot_model_metrics

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

batch_size = 512

train_dataset, test_dataset = generate_balanced_dataset('data/DeepHP')
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True, )
test_loader = DataLoader(test_dataset,  #Subset(test_dataset, np.arange(100)),
                         batch_size=batch_size,
                         shuffle=True,
                         )

model = CnnModel()
# print(model)


cuda:0


In [2]:
print(f"Training starts {datetime.now().isoformat()}")

Training starts 2024-08-09T17:08:56.191841


In [3]:

model = model.to(device)
model, model_metrics = train_classifier(model, train_loader, test_loader, device,
                                        learning_rate=0.001,
                                        weight_decay=0.0001,
                                        max_epochs=50,
                                        checkpoint_every=10,
                                        eval_every=5)

Epoch 1 training:   5%|▍         | 14/304 [00:09<03:11,  1.52it/s]


KeyboardInterrupt: 

In [ ]:
print(f"Training ends {datetime.now().isoformat()}")
for metric in ["accuracy", "precision", "recall", "f1"]:
    if len(model_metrics[f"test_{metric}"]) > 0:
        print(f"Test {metric}:", model_metrics[f"test_{metric}"][-1])

plot_model_metrics(model_metrics)

In [ ]:

torch.save(model.state_dict(), "./model5.bin")


In [ ]:
print(model_metrics)